In [22]:
import os
import duckdb
import fiona
import pandas as pd
from pathlib import Path
from typing import List, Optional

In [2]:
db = r'W:\lwbc\visr\Workarea\moez_labiadh\LEARNING\duckdb\hd_test.db'
conn = duckdb.connect(db)
conn.install_extension('spatial')
conn.load_extension('spatial')

In [27]:

def esri_2_duckdb(
    conn: duckdb.DuckDBPyConnection,
    fgdb_path: Optional[str] = None,
    feature_classes: Optional[List[str]] = None,
    shapefiles: Optional[List[str]] = None,
) -> pd.DataFrame:
    """
    Import ESRI vector data into DuckDB as tables, from either an File Geodatabase
    or standalone Shapefiles. Geometry column is renamed to 'geometry' and an RTREE
    index is created.

    Args:
    -----
      conn : DuckDBPyConnection
        Active DuckDB connection.
      fgdb_path : str, optional
        Filesystem path to the .gdb directory. Required if feature_classes is provided.
      feature_classes : list of str, optional
        Names of GDB layers to import. If None and fgdb_path is set, imports all.
      shapefiles : list of str, optional
        Full paths to .shp files to import.

    Returns:
    ------
      pandas.DataFrame with columns:
        - table_name
        - row_count
        - column_count
        - geometry_column
    """
    tasks = []

    # Prepare GDB layers
    if fgdb_path:
        if feature_classes is None:
            feature_classes = fiona.listlayers(fgdb_path)  # all layers in the GDB
        for lyr in feature_classes:
            tasks.append({
                "source": fgdb_path,
                "driver": "OpenFileGDB",
                "layer": lyr,
                "table": lyr,
            })

    # Prepare Shapefiles
    if shapefiles:
        for shp in shapefiles:
            table_name = Path(shp).stem
            tasks.append({
                "source": shp,
                "driver": "ESRI Shapefile",
                "layer": None,
                "table": table_name,
            })

    total = len(tasks)
    existing = {r[0] for r in conn.execute("SHOW TABLES").fetchall()}

    stats = []
    for i, task in enumerate(tasks, 1):
        tbl = task["table"]
        print(f"\n[{i}/{total}] Importing '{tbl}'…")

        # drop if exists
        if tbl in existing:
            print(" • exists → dropping…")
            conn.execute(f'DROP TABLE IF EXISTS "{tbl}";')

        # build the ST_Read call
        read_sql = [
            f"'{task['source']}'",
            f"allowed_drivers => ['{task['driver']}']"
        ]
        if task["layer"]:
            read_sql.append(f"layer => '{task['layer']}'")

        sql = f"""
            CREATE TABLE "{tbl}" AS
            SELECT * FROM ST_Read(
                {', '.join(read_sql)}
            );
        """
        print(" • reading into DuckDB…")
        conn.execute(sql)

        # rename geometry column
        cols = conn.execute(f"PRAGMA table_info('{tbl}')").fetchall()
        geomcol = next(c[1] for c in cols if c[2].upper() == "GEOMETRY")
        conn.execute(f'ALTER TABLE "{tbl}" RENAME COLUMN {geomcol} TO geometry;')

        # create spatial index
        print(" • creating RTREE index…")
        conn.execute(f"""
            DROP INDEX IF EXISTS idx_geo_{tbl};
            CREATE INDEX idx_geo_{tbl}
              ON "{tbl}" USING RTREE (geometry);
        """)

        # gather stats
        row_count = conn.execute(f'SELECT COUNT(*) FROM "{tbl}";').fetchone()[0]
        stats.append({
            "table_name": tbl,
            "row_count": row_count,
            "column_count": len(cols),
            "geometry_column": "geometry"
        })

    return pd.DataFrame(stats)

In [ ]:
fgdb_path = r'W:\ilmb\dss\projects\GeoBC\Human Disturbance\outputs\2025\Disturbance_1_Raw.gdb'
fc_list = [
    'Power_2_Flooded_Reservoirs_raw', 
    'Power_3_Transmission_Lines_GBA_raw'
]
shp_list = [
    r'W:\lwbc\visr\Workarea\moez_labiadh\TOOLS\SCRIPTS\STATUSING\test_data\aoi_test_7_vvvbig.shp'
]


import_log = esri_2_duckdb( 
    conn = conn,
    fgdb_path= fgdb_path,
    feature_classes= fc_list,
    shapefiles= shp_list
)

import_log


[1/3] Importing 'Power_2_Flooded_Reservoirs_raw'…
 • exists → dropping…
 • reading into DuckDB…
 • creating RTREE index…

[2/3] Importing 'Power_3_Transmission_Lines_GBA_raw'…
 • exists → dropping…
 • reading into DuckDB…
 • creating RTREE index…

[3/3] Importing 'aoi_test_7_vvvbig'…
 • exists → dropping…
 • reading into DuckDB…
 • creating RTREE index…


,table_name,row_count,column_count,geometry_column
0,Power_2_Flooded_Reservoirs_raw,41,31,geometry
1,Power_3_Transmission_Lines_GBA_raw,2002,22,geometry
2,aoi_test_7_vvvbig,1,21,geometry
